# Evaluate the Results

In [1]:
import transformers
import torch
import pandas as pd
import datasets

from torch.utils.tensorboard import SummaryWriter

In [2]:
# define run name
run_name = "NCG_RUN_1"

# define model for tokenizer
model_name = "codellama/CodeLlama-7b-hf"

# dataset import folder
export_folder = "./dataset/" + run_name + "/"

# model save path
model_save_path = "./models/" + run_name + "/"

# model checkpoint path
model_checkpoint_path = "./checkpoints/" + run_name + "/"

In [3]:
## Test loading model and inference with that model

# load quantization config for 4bit quantization -> must be same as training
quantization_config = transformers.BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)

# load model from model_save_path with quantization config
model = transformers.AutoModelForCausalLM.from_pretrained(model_save_path, quantization_config=quantization_config, low_cpu_mem_usage=True)

# optional: load model from checkpoint
# model = transformers.AutoModelForCausalLM.from_pretrained("./output/bigRun/checkpoint-1000", quantization_config=quantization_config, low_cpu_mem_usage=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# load tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

# add pad token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [8]:
### test inference
# prompt = "{ \"@class\" : \"nitrox.dlc.mirror.model.EnumModel\", \"typeName\" :"
# prompt = "{ \"@class\" : \"nitrox.dlc.mirror.model.AggregateRootModel\",\""
prompt = "§ {"
# prompt = "{"

inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(inputs.input_ids, max_length=300, do_sample=True, temperature=0.1)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


§ {name": "VAR_name", "type": {"@class": "nitrox.dlc.mirror.model.AssertedContainableTypeModel", "typeName": "VAR_typeName", "domainType": "NON_DOMAIN", "assertions": [], "hasOptionalContainer": false, "hasCollectionContainer": false, "hasListContainer": false, "hasSetContainer": false, "hasStreamContainer": false, "containerTypeName": "VAR_containerTypeName", "containerAssertions": []}}, "accessLevel": "PACKAGE", "declaredByTypeName": "VAR_declaredByTypeName", "modifiable": true, "publicReadable": true, "publicWriteable": false, "static": false}, "concurrencyVersionField": {"@class": "nitrox.dlc.mirror.model.FieldModel", "name": "VAR_name", "type": {"@class": "nitrox.dlc.mirror.model.AssertedContainableTypeModel", "typeName": "VAR_typeName", "domainType": "NON_DOMAIN", "assertions": [], "hasOptionalContainer": false, "hasCollectionContainer": false, "hasListContainer": false, "hasSetContainer": false, "hasStreamContainer": false, "containerTypeName": "VAR


In [6]:
prompt = "§ {\""

inputs = tokenizer(prompt, return_tensors="pt")
temp = [0.1, 0.3, 0.5, 0.7, 0.9]
outputs_temp = []
for i in range(len(temp)):
    outputs_temp.append(model.generate(inputs.input_ids, max_length=150, do_sample=True, temperature=temp[i]))



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

In [7]:
for i in range(len(temp)):
    print("Temperature: ", temp[i])
    print(tokenizer.decode(outputs_temp[i][0], skip_special_tokens=True))

Temperature:  0.1
§ {"@assertions": [], "publishedEventTypeNames": "VAR_publishedEventTypeNames", "listenedEventTypeName": null, "getter": false, "setter": false}, "setter": false}, "collectionContainerTypeName": null, "listContainerTypeName": "VAR_listContainerTypeName", "setContainerTypeName": "VAR_setContainerTypeName", "enumContainerTypeName": "VAR_enumContainerTypeName", "inheritanceContainerTypeNames": "VAR_inheritanceContainerTypeNames", "allInterfaceTypeNames": "VAR_allInterfaceTypeNames"} §AccessLevel": "PUBLIC", "typeName": "VAR_type
Temperature:  0.3
§ {"@assertions": [], "publishedEventTypeNames": "VAR_publishedEventTypeNames", "listenedEventTypeName": null, "getter": false, "setter": false}, "setter": false}, "collectionContainerTypeName": null, "listContainerTypeName": "VAR_listContainerTypeName", "setContainerTypeName": "VAR_setContainerTypeName", "containerAssertions": []}, "accessLevel": "PRIVATE", "declaredByTypeName": "VAR_declaredByTypeName", "modifiable": true, "pu